In [ ]:
!pip install folium geopy rtree matplotlib mapclassify 
!pip install geopandas


In [ ]:
import folium
import geopandas as gpd
import pandas as pd
import os
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, Fullscreen, MousePosition

In [ ]:
# Barrios Dataset
barrios = load_shapefile("barrios", ["NOMBRE", "geometry"], index="NOMBRE")
distritos = load_shapefile("distritos", ["DISTRI_MAY", "geometry"], rename={"DISTRI_MAY": "NOMBRE"}, index="NOMBRE")


In [ ]:
def load_shapefile(name, cols=None, index=None, rename=None):
    path = os.path.abspath(f"./data/{name}.zip")
    df = gpd.read_file(f'zip://{path}')

    if cols:
        df = df[cols]
    if rename:
        df = df.rename(columns=rename)
    if index:
        df = df.set_index(index)
    return df.to_crs(epsg=4326)
    

def add_barrios(m):
    for name, r in barrios.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {'fillColor': 'orange'})
        folium.Popup(f"<b>Barrio:</b> <span style='white-space: nowrap;'>{name}</span>").add_to(geo_j)
        geo_j.add_to(m)
    m



def get_map(withbarrios=True):
    #tiles maps: https://github.com/python-visualization/folium/tree/main/folium/templates/tiles
    # Create a base map
    m = folium.Map(location=[40.422, -3.696], tiles='stamentoner', zoom_start=12)
    m.add_child(Fullscreen())
    m.add_child(MousePosition())
    
    if withbarrios:
        add_barrios(m)
    
    return m

In [ ]:
# Barrios and distritos Dataset
barrios = load_shapefile("barrios", ["NOMBRE", "geometry"], index="NOMBRE")
distritos = load_shapefile("distritos", ["DISTRI_MAY", "geometry"], rename={"DISTRI_MAY": "NOMBRE"}, index="NOMBRE")


In [ ]:
# Load all Datasets
bus_interurbano_estaciones = load_shapefile("bus-interurbano-estaciones", 
                                            cols=["IDPOSTE", "geometry"], 
                                            rename={"IDPOSTE": "NOMBRE"},
                                            index="NOMBRE")
bus_interurbano_estaciones["TYPE"] = "bus-interurbano-estaciones"
bus_interurbano_estaciones["latitude"] = bus_interurbano_estaciones.apply(lambda x: x['geometry'].y, axis=1)
bus_interurbano_estaciones["longitude"] = bus_interurbano_estaciones.apply(lambda x: x['geometry'].x, axis=1)

bus_urbano_madrid_estaciones = load_shapefile("bus-urbano-madrid-estaciones", 
                                              cols=["IDPOSTE", "geometry"], 
                                              rename={"IDPOSTE": "NOMBRE"},
                                              index="NOMBRE")
bus_urbano_madrid_estaciones["TYPE"] = "bus-urbano-madrid-estaciones"
bus_urbano_madrid_estaciones["latitude"] = bus_urbano_madrid_estaciones.apply(lambda x: x['geometry'].y, axis=1)
bus_urbano_madrid_estaciones["longitude"] = bus_urbano_madrid_estaciones.apply(lambda x: x['geometry'].x, axis=1)

bus_urbano_no_madrid_estaciones = load_shapefile("bus-urbano-no-madrid-estaciones", 
                                                 cols=["IDPOSTE", "geometry"], 
                                                 rename={"IDPOSTE": "NOMBRE"},
                                                 index="NOMBRE")
bus_urbano_no_madrid_estaciones["TYPE"] = "bus-urbano-no-madrid-estaciones"
bus_urbano_no_madrid_estaciones["latitude"] = bus_urbano_no_madrid_estaciones.apply(lambda x: x['geometry'].y, axis=1)
bus_urbano_no_madrid_estaciones["longitude"] = bus_urbano_no_madrid_estaciones.apply(lambda x: x['geometry'].x, axis=1)
bus_urbano_estaciones = pd.concat([bus_urbano_madrid_estaciones, bus_urbano_no_madrid_estaciones])

cercanias_estaciones = load_shapefile("cercanias-estaciones", 
                                      cols=["DENOMINACI", "geometry"], 
                                      rename={"DENOMINACI": "NOMBRE"}, 
                                      index="NOMBRE")
cercanias_estaciones["TYPE"] = "cercanias-estaciones"
cercanias_estaciones["latitude"] = cercanias_estaciones.apply(lambda x: x['geometry'].y, axis=1)
cercanias_estaciones["longitude"] = cercanias_estaciones.apply(lambda x: x['geometry'].x, axis=1)

metro_estaciones = load_shapefile("metro-estaciones", 
                                  cols=["DENOMINACI", "geometry"], 
                                  rename={"DENOMINACI": "NOMBRE"}, 
                                  index="NOMBRE")
metro_estaciones["TYPE"] = "metro-estaciones"
metro_estaciones["latitude"] = metro_estaciones.apply(lambda x: x['geometry'].y, axis=1)
metro_estaciones["longitude"] = metro_estaciones.apply(lambda x: x['geometry'].x, axis=1)

metro_ligero_estaciones = load_shapefile("metro-ligero-estaciones", 
                                         cols=["DENOMINACI", "geometry"], 
                                         rename={"DENOMINACI": "NOMBRE"}, 
                                         index="NOMBRE")
metro_ligero_estaciones["TYPE"] = "metro-ligero-estaciones"
metro_ligero_estaciones["latitude"] = metro_ligero_estaciones.apply(lambda x: x['geometry'].y, axis=1)
metro_ligero_estaciones["longitude"] = metro_ligero_estaciones.apply(lambda x: x['geometry'].x, axis=1)

#bus_interurbano_estaciones, bus_urbano_estaciones
#cercanias_estaciones, metro_estaciones, metro_ligero_estaciones
#pd.concat([A, B])



In [ ]:
#bus_interurbano_estaciones, bus_urbano_estaciones
#cercanias_estaciones, metro_estaciones, metro_ligero_estaciones
#pd.concat([A, B])

df = bus_urbano_estaciones

# Show bicimad stations as heatmap
m = get_map()
HeatMap(data=df[['latitude', 'longitude']], radius=20).add_to(m)
m

In [ ]:
#bus_interurbano_estaciones, bus_urbano_estaciones
#cercanias_estaciones, metro_estaciones, metro_ligero_estaciones
#pd.concat([A, B])

df = bus_urbano_estaciones

estaciones_barrios = gpd.sjoin(barrios, df).reset_index()
estaciones_barrios = estaciones_barrios[["NOMBRE"]].groupby(['NOMBRE'])['NOMBRE'].count().squeeze()


# Create a base map
m = get_map(withbarrios=False)


# Add a choropleth map to the base map
cp = Choropleth(geo_data=barrios.__geo_interface__,
           data=estaciones_barrios, 
           key_on="feature.id", 
           fill_color="OrRd", 
           legend_name='Bicimad Station docks per district',
           nan_fill_color = "LightGrey"
          ).add_to(m)

# looping thru the geojson object and adding a new property(unemployment)
# and assigning a value from our dataframe   
for s in cp.geojson.data['features']:
    s['properties']['NOMBRE'] = s['id']
    s['properties']['total'] = int(estaciones_barrios.get(s['id'], 0))

# and finally adding a tooltip/hover to the choropleth's geojson
folium.GeoJsonTooltip(['NOMBRE', 'total']).add_to(cp.geojson)

folium.LayerControl().add_to(m)

# Display the map
m